In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [2]:
df = pd.read_csv('Breast_cancer_data[1].csv')
df

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0
1,20.57,17.77,132.90,1326.0,0.08474,0
2,19.69,21.25,130.00,1203.0,0.10960,0
3,11.42,20.38,77.58,386.1,0.14250,0
4,20.29,14.34,135.10,1297.0,0.10030,0
...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0
565,20.13,28.25,131.20,1261.0,0.09780,0
566,16.60,28.08,108.30,858.1,0.08455,0
567,20.60,29.33,140.10,1265.0,0.11780,0


In [3]:
#adding a new column for the line intercept and initializing with ones
#inserting a new column as first column
df.insert(loc=0,column='theta0',value=1)
df

,theta0,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,diagnosis
0,1,17.99,10.38,122.80,1001.0,0.11840,0
1,1,20.57,17.77,132.90,1326.0,0.08474,0
2,1,19.69,21.25,130.00,1203.0,0.10960,0
3,1,11.42,20.38,77.58,386.1,0.14250,0
4,1,20.29,14.34,135.10,1297.0,0.10030,0
...,...,...,...,...,...,...,...
564,1,21.56,22.39,142.00,1479.0,0.11100,0
565,1,20.13,28.25,131.20,1261.0,0.09780,0
566,1,16.60,28.08,108.30,858.1,0.08455,0
567,1,20.60,29.33,140.10,1265.0,0.11780,0


In [4]:
#splitting the dataset into training and testing set
r = df.shape[0]
c = df.shape[1]
#df = np.array(df)
idx = 8*r//10
x_train = df.iloc[:idx,:c-1]
y_train = df.iloc[:idx,-1]
x_test  = df.iloc[idx:,:c-1]
y_test  = df.iloc[idx:,-1]
print("x training data:")
print(x_train,end="\n\n")
print("y training data:")
print(y_train,end="\n\n")
print("x testing data:")
print(x_test,end="\n\n")
print("y testing data:")
print(y_test)

x training data:
     theta0  mean_radius  mean_texture  mean_perimeter  mean_area  \
0         1        17.99         10.38          122.80     1001.0   
1         1        20.57         17.77          132.90     1326.0   
2         1        19.69         21.25          130.00     1203.0   
3         1        11.42         20.38           77.58      386.1   
4         1        20.29         14.34          135.10     1297.0   
..      ...          ...           ...             ...        ...   
450       1        11.87         21.54           76.83      432.0   
451       1        19.59         25.00          127.70     1191.0   
452       1        12.00         28.23           76.77      442.5   
453       1        14.53         13.98           93.86      644.2   
454       1        12.62         17.15           80.62      492.9   

     mean_smoothness  
0            0.11840  
1            0.08474  
2            0.10960  
3            0.14250  
4            0.10030  
..              

In [5]:
#normalizing the data
for c in x_train.columns:
    if c!='theta0':
        min_train = x_train[c].min()
        max_train = x_train[c].max()
        min_test = x_test[c].min()
        max_test = x_test[c].max()
        x_train[c]=(x_train[c]-min_train)/(max_train-min_train)
        x_test[c]=(x_test[c]-min_test)/(max_test-min_test)
print(x_train,end="\n\n")
print(x_test)

     theta0  mean_radius  mean_texture  mean_perimeter  mean_area  \
0         1     0.521037      0.022658        0.545989   0.364042   
1         1     0.643144      0.272574        0.615783   0.502017   
2         1     0.601496      0.390260        0.595743   0.449798   
3         1     0.210090      0.360839        0.233501   0.102993   
4         1     0.629893      0.156578        0.630986   0.489705   
..      ...          ...           ...             ...        ...   
450       1     0.231388      0.400068        0.228319   0.122479   
451       1     0.596763      0.517078        0.579849   0.444704   
452       1     0.237541      0.626310        0.227904   0.126937   
453       1     0.357281      0.144403        0.346002   0.212566   
454       1     0.266884      0.251606        0.254509   0.148334   

     mean_smoothness  
0           0.680010  
1           0.270471  
2           0.572941  
3           0.973233  
4           0.459788  
..               ...  
450       

In [6]:
#converting the training and testing datasets into numpy arrays
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [7]:
# Using hinge loss function
# def hypothesis(x,theta):
#     return np.dot(x,theta)


def grad_descent(x,y,reg,alpha,epoch):
    r=x.shape[0]
    c=x.shape[1]
    theta = np.ones((c,1))
    min_cost=None
    min_theta=[]
    cst=[] #to store costs
    
    for itr in range(epoch):
        cost = reg*np.dot(theta.T,theta)
        for i in range(r):
            if y[i]*np.dot(x[i],theta)>=1:
                theta = theta-alpha*(2*reg*theta)
            else:
                for j in range(1,c):
                    theta[j]-=alpha*(2*reg*theta[j]-y[i]*x[i,j])
                theta[0]-=alpha*(x[i,0])
                cost += 1-y[i]*np.dot(x[i],theta)
                
        cst.append(cost)
        #print(theta)
        #print(cost)
        if min_cost is None or min_cost>cost:
            min_cost=cost
            min_theta=theta
    return min_theta,cost

def plotcost(cost):
    plt.plot(cost)
    plt.xlabel('no. of iterations')
    plt.ylabel('cost')
    
def calAccuracy(pred,y_):
    correct=0
    for i in range(pred.shape[0]):
        if pred[i]==y_[i]:
            correct+=1
    return correct*100/pred.shape[0]

def predict(x_,y_,theta):
    py = np.dot(x_,theta)
    for i in range(py.shape[0]):
        if py[i]>=1:
            py[i]=1
        else:
            py[i]=0
    return py

def showPredictions(pred,y_):
    for i in range(pred.shape[0]):
        print("predicted: ",py[i],end='  ')
        print("actual: ",y_[i])

In [8]:
alpha = 0.5
epoch = 1000
reg = 0.1 # regularization parameter
theta,cost=grad_descent(x_train,y_train,reg,alpha,epoch)
print("costs: ",cost)
print(theta)

costs:  [[5.22640445e+09]]
[[-2.27499000e+05]
 [ 1.00993166e+00]
 [ 1.14615313e+00]
 [ 9.76708027e-01]
 [ 5.74139347e-01]
 [ 1.14690020e+00]]


In [9]:
pred_y = predict(x_test,y_test,theta)
accuracy = calAccuracy(pred_y,y_test)
print(f"accuracy of model: {accuracy}%")

accuracy of model: 22.80701754385965%


In [10]:
alpha = 0.01
epoch = 1000
reg = 0.01 # regularization parameter
theta,cost=grad_descent(x_train,y_train,reg,alpha,epoch)
pred_y = predict(x_test,y_test,theta)
accuracy = calAccuracy(pred_y,y_test)
print(f"accuracy of model: {accuracy}%")
print(theta)

accuracy of model: 22.80701754385965%
[[-4.54800463e+03]
 [ 7.24273590e+00]
 [ 7.63581463e+00]
 [ 7.01193870e+00]
 [ 4.00717028e+00]
 [ 1.06413000e+01]]


In [11]:
alpha = 0.1
epoch = 10000
reg = 0.001 # regularization parameter
theta,cost=grad_descent(x_train,y_train,reg,alpha,epoch)
pred_y = predict(x_test,y_test,theta)
accuracy = calAccuracy(pred_y,y_test)
print(f"accuracy of model: {accuracy}%")
print(theta)

accuracy of model: 22.80701754385965%
[[-4.54999000e+05]
 [ 7.24273590e+01]
 [ 7.63581463e+01]
 [ 7.01193870e+01]
 [ 4.00717028e+01]
 [ 1.06413000e+02]]
